In [ ]:
# -*- coding: utf-8 -*-
"""This script process excel files from the scenarios and base case and extract data from the selected impact categories and make a csv file. This is the first step of scenario comparison. The script can process multiple files.
"""

# Block 1: Setup and Function Definitions
# Install required libraries and import dependencies
!pip install pandas numpy matplotlib seaborn openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from google.colab import files

def process_excel_file(filename):

    print(f"\nProcessing {filename}...")

    # Read the Excel file, skipping the first 15 rows
    df = pd.read_excel(filename, header=15)

    # Get the name of the 4th column (index 3)
    column_to_remove = df.columns[3]
    print(f"Removing column at position 4: {column_to_remove}")

    # Remove the 4th column
    df = df.drop(column_to_remove, axis=1)

    # Define common process columns
    processes = [
        'Thin stillage 25% for 1 kg',
        'paccino_02_heating_energy_perLiter',
        'paccino_02_stirring_energy_perLiter',
        'paccino_02_drying_energy_perKg',
        'paccino_02_filtration_energy_perKg'
    ]

    # Find wastewater column dynamically
    wastewater_columns = [col for col in df.columns if
                         isinstance(col, str) and
                         'Wastewater Treatment for 1 kg' in col]

    if wastewater_columns:
        wastewater_column = wastewater_columns[0]
        print(f"Found wastewater column: {wastewater_column}")
    else:
        wastewater_column = None
        print("No wastewater column found")

    # Find sodium hydroxide column for pH adjustment
    sodium_columns = [col for col in df.columns if
                     isinstance(col, str) and
                     'Sodium hydroxide' in col]

    if sodium_columns:
        sodium_column = sodium_columns[0]
        print(f"Found pH adjustment column: {sodium_column}")
    else:
        sodium_column = None
        print("No pH adjustment column found")

    # Update processes list with dynamic columns
    dynamic_processes = processes.copy()
    if wastewater_column:
        dynamic_processes.append(wastewater_column)
    if sodium_column:
        dynamic_processes.append(sodium_column)

    # Remove the CO2 fermentation column if it exists
    co2_columns = [col for col in df.columns if
                  isinstance(col, str) and
                  'CO2 Release of Fermentation' in col]

    if co2_columns:
        for col in co2_columns:
            if col in df.columns:
                df = df.drop(col, axis=1)
                print(f"Removed CO2 fermentation column: {col}")

    # Selected categories to filter by
    selected_categories = [
        'Global warming',
        'Terrestrial acidification',
        'Freshwater eutrophication',
        'Marine eutrophication',
        'Land use',
        'Fossil resource scarcity',
        'Water consumption'
    ]

    # Create new DataFrame with selected categories
    df_selected = df.loc[df['Impact category'].isin(selected_categories)]

    # Create process column subsets for printing
    display_columns = ['Impact category', 'Total']
    for proc in dynamic_processes:
        if proc in df.columns:
            display_columns.append(proc)

    # Print verification
    print(f"\nSelected impact categories for {filename}:")
    print(df_selected[display_columns])

    return df_selected, dynamic_processes


# Upload Excel files and define scenario names
def upload_and_name_files():
    # Upload Excel files
    print("Please upload your Excel files...")
    uploaded = files.upload()  # This will prompt you to upload files

    # Print the names of uploaded files
    print("\nUploaded files:")
    file_paths = list(uploaded.keys())
    for i, filename in enumerate(file_paths):
        print(f"{i+1}. {filename}")

    # Ask user to name each scenario
    scenario_names = {}

    print("\nPlease provide a scenario name for each file (press Enter to use the filename without extension):")
    for filename in file_paths:
        default_name = os.path.splitext(os.path.basename(filename))[0]
        user_input = input(f"Scenario name for '{filename}' [default: {default_name}]: ")

        # Use default name if user doesn't provide one
        scenario_name = user_input.strip() if user_input.strip() else default_name
        scenario_names[filename] = scenario_name

    return file_paths, scenario_names

# Process files with selected scenario names
file_paths, scenario_names = upload_and_name_files()
all_results = {}

# Process each file with its scenario name
for file_path in file_paths:
    if file_path in scenario_names:
        scenario_name = scenario_names[file_path]
        print(f"\nProcessing scenario '{scenario_name}' from file: {file_path}")

        try:
            # Process the file
            selected_df, dynamic_procs = process_excel_file(file_path)

            # Store results with scenario name as prefix
            all_results[scenario_name] = {
                'file': file_path,
                'all_impacts': selected_df,
                'processes': dynamic_procs
            }

        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")
    else:
        print(f"\nNo scenario name defined for file: {file_path}")


# Save Results to CSV and Download

output_folder = 'lca_results'
!mkdir -p {output_folder}

saved_files = []

# Save all impacts dataframe for each scenario
for scenario_name, result_dict in all_results.items():
    # Clean scenario name for filenames (replace spaces with underscores)
    clean_name = scenario_name.replace(' ', '_')

    # Save all impacts dataframe
    all_impacts_filename = f"{output_folder}/{clean_name}_all_impacts.csv"
    result_dict['all_impacts'].to_csv(all_impacts_filename, index=False)
    saved_files.append(all_impacts_filename)
    print(f"Saved: {all_impacts_filename}")

print(f"\nSaved {len(saved_files)} CSV files in the '{output_folder}' folder.")

# Download all CSV files to the user's computer
print("\nDownloading CSV files to your computer...")
for filepath in saved_files:
    files.download(filepath)

print("Process completed successfully!")

Please upload your Excel files...


Saving Basecase_model5_Sheet1.csv to Basecase_model5_Sheet1.csv
Saving water_reduc_25_Sheet1.csv to water_reduc_25_Sheet1.csv
Saving water_reduc_50_Sheet1.csv to water_reduc_50_Sheet1.csv
Saving water_reduc_75_Sheet1.csv to water_reduc_75_Sheet1.csv

Uploaded files:
1. Basecase_model5_Sheet1.csv
2. water_reduc_25_Sheet1.csv
3. water_reduc_50_Sheet1.csv
4. water_reduc_75_Sheet1.csv

Please provide a scenario name for each file (press Enter to use the filename without extension):


KeyboardInterrupt: Interrupted by user